In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.options import Options
import regex as re
import requests
import time
import numpy as np
import pandas as pd
import math

In [15]:
pip install js2py

     |████████████████████████████████| 1.0 MB 3.8 MB/s eta 0:00:01
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25998 sha256=c3e78b68d1eb356074dba70edd2810569de85553fb3f0b3d874f55e1666f4856
  Stored in directory: /Users/samehrlich/Library/Caches/pip/wheels/d5/88/34/ccb5bb40eb3178a134eb293e6c363928c5bcfba0b91031db76
Successfully built pyjsparser
Note: you may need to restart the kernel to use updated packages.


In [34]:
import setup.py from './fetch.0.0/setup.py'

SyntaxError: invalid syntax (<ipython-input-34-21a06570e22a>, line 1)

In [31]:
pwd

'/Users/samehrlich'

In [29]:
import js2py
import fetch

ModuleNotFoundError: No module named 'fetch'

In [12]:
def getPageResponceData(town_r,city_r,state_r,chromeDriverPath):
    
    if(city_r.find(" ")!= -1):
        city_r[city_r.find(" ")]="-"
    
    if(state_r.find(" ")!= -1):
        state_r[state_r.find(" ")]="-"
        
    if(town_r.find(" ")!= -1):
        town_r[town_r.find(" ")]="-"
    if (len(town_r)>0):
            address =town_r+'-'+city_r+'-'+state_r+'/'
    if (len(city_r)>0):
            address =city_r+'-'+state_r+'/'
    elif (len(state_r)>0):
             address =state_r+'/'
    else:
        return ("There was an error with the requested address")

#this gets the responce data from each page
    with requests.session() as s:
        headers = {
    'authority': 'www.zillow.com',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.zillow.com/los-angeles-ca/2_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%22%2C%22mapBounds%22%3A%7B%22west%22%3A-118.88620504589845%2C%22east%22%3A-117.93726095410157%2C%22south%22%3A33.54554919445917%2C%22north%22%3A34.49481997322805%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D',
    'accept-language': 'en-US,en;q=0.9',
    }
       #grabs the number of page results
        page = 1
        uRL = 'https://www.zillow.com/'+address+ f'{page}_p/'
        request = s.get(uRL,headers=headers)
        startpage = bs(request.content,'html.parser')
        for i in startpage:
            numberOfPages = startpage.find(class_="Text-c11n-8-37-0__aiai24-0 eBcXID")
            numberOfPages = str(numberOfPages)
            numberOfPages = numberOfPages.replace('<span class="Text-c11n-8-37-0__aiai24-0 eBcXID">Page <!-- -->1<!-- --> of <!-- -->',"")
            numberOfPages = numberOfPages.replace("</span>","")

    end_page = int(numberOfPages)
    url =''
    url_list = []

    #this while loop populates the url_list with all the pages we want to get
    while page <= end_page:
        url = 'https://www.zillow.com/'+address+ f'{page}_p/'
        url_list.append(url)
        page+=1

    request=''
    request_list = []
    # Tell Selenium to get the URL you're interested in.

    for url in url_list:
        request =seleniumRequest(url,chromeDriverPath)
      
        request_list.append(request)

    soup = ''
    soup_list = []

    for request in request_list:
        soup = bs(request,'html.parser')
        soup_list.append(soup)
    return soup_list;  
   


       

In [13]:
def cleanPageSoupData(PageData):
    df_list = []
    x = 1
    
    for page in PageData:
        df = pd.DataFrame()
        
        for i in page:
            
            listing = page.find_all(class_="list-card list-card-additional-attribution list-card-additional-attribution-space list-card_not-saved")
            links = page.find_all('a', class_="list-card-link list-card-link-top-margin")

            price = list(page.find_all (class_='list-card-price'))
            beds = list(page.find_all("ul", class_="list-card-details"))
            details = page.find_all ('div', {'class': 'list-card-details'})
            home_type = page.find_all ('div', {'class': 'list-card-footer'})
            last_updated = page.find_all ('div', {'class': 'list-card-top'})
            brokerage = list(page.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
        
           
            df['prices'] = price
            df['prices'] = df['prices'].astype('str')
            df['prices'] = df['prices'].replace('<div class="list-card-price">', ' ', regex=True)
            df['prices'] = df['prices'].replace('</div>', ' ', regex=True)
            df['beds'] = beds

        x+=1
        urls = []
        addresses=[]
        for link in links:

            href = link.get('href')
            address = link.find('address')
            
            addresses.append(address)
            urls.append(href)

        df['links'] = urls
        df['links'] = df['links'].astype('str')
        df['links'] = df['links'].replace('<a class="list-card-link list-card-link-top-margin" href="', ' ', regex=True)
        df['links'] = df['links'].replace('" tabindex="0"></a>', ' ', regex=True)
        df['address'] = addresses
        df['address'] = df['address'].astype('str')
        df['address'] = df['address'].replace('<address class="list-card-addr">', ' ', regex=True)
        df['address'] = df['address'].replace('</address>', ' ', regex=True)
        df_list.append(df)
    return df_list

In [14]:
def makePretty(df_list,filePath):
    
    
    df = pd.concat(df_list).reset_index().drop(columns='index')
    df['beds']=df['beds'].astype('str')
    
    #filter unwanted property types
    df = df[~df['beds'].str.contains("Land for sale")]

    #remove html tags from beds column
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> bds</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> ba</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!-- -->Foreclosure</abbr>', '- Foreclosure', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> bd</abbr></li><li class="">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> Foreclosure</abbr>', '- Foreclosure', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> sqft</abbr></li><li class="list-card-statusText">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> acres lot</abbr></li><li class="list-card-statusText">', ' ', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('</li></ul>', '', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('--', '0', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('Multi-family', 'Multifamily', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace(' for sale', '', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!0 0>Auction</abbr>', '- Auction', regex=True)
    df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!0 0>Pending</abbr>', '- Pending', regex=True)
    
    df[['beds','type']]=df.beds.apply(
    lambda x: pd.Series(str(x).split('-')))
    
    df[['beds', 'baths', 'sq_feet']] = df.beds.str.split(expand=True)
    df['prices'] = df['prices'].astype('str')
    
    df['prices'] = df['prices'].replace('<div class="list-card-price">', ' ', regex=True)
    df['prices'] = df['prices'].replace('</div>', ' ', regex=True)
    writer = pd.ExcelWriter(filePath)
    # write dataframe to excel
    df.to_excel(writer)
    # save the excel
    writer.save()
    print('DataFrame is written successfully to Excel File.')
    return df



In [21]:
def seleniumRequest(url,chromeDriverPath):
    chrome_options = Options()
    chrome_options.add_argument("--window-size=800,850")
    browser = webdriver.Chrome(chromeDriverPath,options=chrome_options) 
    
    browser.get(url)

    result = browser.execute_script("window.scrollBy(0,window.innerHeight);return [window.innerHeight,document.body.scrollHeight];");


    for i in range(0,math.ceil(result[1]/result[0])-4):
        browser.execute_script("window.scrollBy(0,window.innerHeight);");
        time.sleep(.25)
    
    returnValue = browser.page_source
    browser.close()
    return returnValue

In [22]:
def runZillowScraper(town,city,state,filePath,chromDriverPath):
    soupList =getPageResponceData(town,city,state,chromDriverPath)
    df_list =cleanPageSoupData(soupList)
    prettyList = makePretty(df_list,filePath)

# This is the path where the resulting excel document will be created
path ='/Users/samehrlich/Downloads/Result.xlsx'
#If the location does not have  town leave this as '' and only right the city and state
town =''
#If the city has a space put - between the space
city ='palo-alto'
state='ca'

#You need to download the chromedrive in the ziped fold and copy the path were you downloaded it here
chromDriverPath='/Users/samehrlich/chromedriver'
runZillowScraper(town,city,state,path,chromDriverPath)
print('Check you passed to the PATH variable to see results')

DataFrame is written successfully to Excel File.
Check you passed to the PATH variable to see results


In [17]:
#example 1
#a js command
js1 = 'console.log( "Hello World!" )'
res1 = js2py.eval_js(js1)

'Hello World!'


In [27]:
zillowfunction = '''function getZillowData(town_r,city_r,state_r){
    //clean through address information 
//   These are local variables used to help gather the list of url that we want to request
        fetch('https://www.zillow.com/stanford-ca/', {
            headers: {
                'Accept': '*/*',
                'Accept-Encoding': 'gzip, deflate, br',
                'Host': 'www.zillow.com',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1 Safari/605.1.15',
                'Accept-Language': 'en-us',
                'Connection': 'keep-alive',
            }
        }).then(function(result){console.log(result.headers)})
        
        fetch('https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Palo%20Alto%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.37393699121095%2C%22east%22%3A-121.89740500878908%2C%22south%22%3A37.219977979124586%2C%22north%22%3A37.5429817292495%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A26374%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D&wants=\{%22cat1%22:\[%22mapResults%22\]\}', {

// "https://www.zillow.com/menlo-park-ca/                                                                                                                                                                                                                                           2C%22mapZoom%22%3A13                                                                                                    
headers: {
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Host': 'www.zillow.com',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1 Safari/605.1.15',
            'Accept-Language': 'en-us',
            'Connection': 'keep-alive',
        }
}).then(function(result){
        result.text().then(function(result1){
                
            // console.log(result1)
           
            var ZillowResponse = JSON.parse(result1).cat1
            var resultsTitle = ZillowResponse.searchList.listResultsTitle
            var Listings = ZillowResponse.searchResults.mapResults
            var listingCount = ZillowResponse.searchList.totalResultCount
            var ListingInfo = Listings.map(function(house){
                var listing ={
                    info: house.hdpData
                }
                return listing
            })
            // console.log(result1)
            var homeInfo = Object.values(Listings).map(function(listing){return listing})
            // console.log(homeInfo[0])
            var cleanedListing = Listings.map(function(house,indx){
                var beds = house.beds
                var baths = house.baths
                var address = house.address
                if(beds === null){
                    beds = '--'
                }
                if(baths === null){
                    baths = '--'
                }
                const regex = /([^:\/\s]+)/;
                if(address === "--"){
                    address = house.detailUrl.slice(13).match(regex)[0]
                }
                if(house.unitCount){
                    if(house.hdpData){
                        var newListing = {
                        
                        
                            homeType:house.hdpData.homeInfo.homeType,
                            price: house.price,
                            beds: house.minBeds,
                            baths: house.minBaths,
                            area: house.minArea,
                            image:house.imgSrc,
                            unitCount:house.unitCount,
                            address: address,
                            statusType:house.statusType,
                            statusText:house.statusText,
                            link:"www.zillow.com/"+house.detailUrl,
        
            
                            
                            index:indx
                            
                        }
                        return newListing;
                    }
                    var newListing = {
                        
                        
                        homeType:"--",
                        price: house.price,
                        beds: house.minBeds,
                        baths: house.minBaths,
                        area: house.minArea,
                        image:house.imgSrc,
                        unitCount:house.unitCount,
                        address: address,
                        statusType:house.statusType,
                        statusText:house.statusText,
                        link:"www.zillow.com/"+house.detailUrl,
        
                        
                        index:indx
                        
                    }
                    return newListing;
                }
                if(house.hdpData){
                    var newListing = {
                    
                    
                        homeType:house.hdpData.homeInfo.homeType,
                        price: house.price,
                        beds: beds,
                        baths: baths,
                        area: house.area,
                        image:house.imgSrc,
                        unitCount:"--",
                        address: address,
                        statusType:house.statusType,
                        statusText:house.statusText,
                        link: "www.zillow.com/"+house.detailUrl,
        
        
                        
                        index:indx
                        
                    }
                    return newListing;
                }
                var newListing ={
                        homeType:"--",
                        price: house.price,
                        beds: beds,
                        baths: baths,
                        area: house.area,
                        statusType:house.statusType,
                        statusText:house.statusText,
                        image:house.imgSrc,
                        unitCount:"--",
                        address: address,
                        link:"www.zillow.com/"+house.detailUrl,
                        // info: house.hdpData,
                        index:indx
            
                
                        }
                       
                        
                        return newListing
            })
            const df = new DataFrame({
                column1: cleanedListing.map(function(listing){return listing.price}), // <------ A column
                column2: cleanedListing.map(function(listing){return listing.address}),
                column3: cleanedListing.map(function(listing){return listing.homeType}),
                column4: cleanedListing.map(function(listing){return listing.beds}), // <------ A column
                column5: cleanedListing.map(function(listing){return listing.baths}),
                column6: cleanedListing.map(function(listing){return listing.area}),
                column7: cleanedListing.map(function(listing){return listing.statusType}),
                column8: cleanedListing.map(function(listing){return listing.link}),
                column9: cleanedListing.map(function(listing){return listing.unitCount}),
            }, ['Price', 'Address','homeType','beds','baths','sq_feet','Status','Link','Unit Count']);
            var x = df.toCSV(true, '/Users/samehrlich/Desktop/final-app/client/src/zillowResult.csv')
            console.log(df)
            // convertToExcel()
           
        
        })
    }
)
}'''
function1 = js2py.eval_js(zillowfunction)
df = function1("","menlo-park",'ca')

JsException: ReferenceError: fetch is not defined